In [1]:
import pandas as pd
import requests
from tqdm.auto import tqdm
tqdm.pandas()

In [2]:
def next_monday(date):
    return pd.date_range(start=date, end=date + pd.offsets.Day(6), freq='W-MON')[0]

# Setting

In [3]:
test_date = pd.to_datetime('2021-05-10')

In [4]:
models_to_exclude = ['COVIDhub-ensemble', 'COVIDhub-trained_ensemble', 
                     'CU-nochange', 'CU-scenario_high', 'CU-scenario_low', 'CU-scenario_mid']

locations_to_exclude = ["60", "66", "69", "72", "74", "78"]

# DC,11,District of Columbia
# AS,60,American Samoa
# GU,66,Guam
# MP,69,Northern Mariana Islands
# PR,72,Puerto Rico
# UM,74,U.S. Minor Outlying Islands
# VI,78,Virgin Islands

In [5]:
dtype={'target': str, 'location': str, 'type': str, 'quantile': float, 'value': float}
parse_dates=['forecast_date', 'target_end_date']

# Load Files

In [6]:
url = "https://api.github.com/repos/reichlab/covid19-forecast-hub/git/trees/master?recursive=1"
r = requests.get(url)
res = r.json()

In [7]:
files = [file["path"] for file in res["tree"] if (file["path"].startswith('data-processed/') and file["path"].endswith('.csv'))]

In [8]:
files[-5:]

['data-processed/epiforecasts-ensemble1/2021-04-12-epiforecasts-ensemble1.csv',
 'data-processed/epiforecasts-ensemble1/2021-04-19-epiforecasts-ensemble1.csv',
 'data-processed/epiforecasts-ensemble1/2021-04-26-epiforecasts-ensemble1.csv',
 'data-processed/epiforecasts-ensemble1/2021-05-03-epiforecasts-ensemble1.csv',
 'data-processed/epiforecasts-ensemble1/2021-05-10-epiforecasts-ensemble1.csv']

In [9]:
df_files = pd.DataFrame({'filename':files})

df_files['model'] = df_files.filename.apply(lambda f: f.split('/')[1])

df_files['forecast_date'] = df_files.filename.apply(lambda f: f.split('/')[2][:10])
df_files.forecast_date = pd.to_datetime(df_files.forecast_date)

df_files['timezero'] = df_files.forecast_date.apply(next_monday)

df_files = df_files[~df_files.model.isin(models_to_exclude)]

In [10]:
df_files

,filename,model,forecast_date,timezero
0,data-processed/AIpert-pwllnod/2020-12-21-AIper...,AIpert-pwllnod,2020-12-21,2020-12-21
1,data-processed/AIpert-pwllnod/2020-12-28-AIper...,AIpert-pwllnod,2020-12-28,2020-12-28
2,data-processed/AIpert-pwllnod/2021-01-04-AIper...,AIpert-pwllnod,2021-01-04,2021-01-04
3,data-processed/AIpert-pwllnod/2021-01-11-AIper...,AIpert-pwllnod,2021-01-11,2021-01-11
4,data-processed/AIpert-pwllnod/2021-01-18-AIper...,AIpert-pwllnod,2021-01-18,2021-01-18
...,...,...,...,...
3362,data-processed/epiforecasts-ensemble1/2021-04-...,epiforecasts-ensemble1,2021-04-12,2021-04-12
3363,data-processed/epiforecasts-ensemble1/2021-04-...,epiforecasts-ensemble1,2021-04-19,2021-04-19
3364,data-processed/epiforecasts-ensemble1/2021-04-...,epiforecasts-ensemble1,2021-04-26,2021-04-26
3365,data-processed/epiforecasts-ensemble1/2021-05-...,epiforecasts-ensemble1,2021-05-03,2021-05-03


### Test Data

In [11]:
VALID_TARGETS = [f"{_} wk ahead inc death" for _ in range(1, 5)] + \
                [f"{_} wk ahead cum death" for _ in range(1, 5)] + \
                [f"{_} wk ahead inc case" for _ in range(1, 5)]

In [12]:
df_test_files = df_files[df_files.timezero == test_date]

In [13]:
df_test_files.head()

,filename,model,forecast_date,timezero
20,data-processed/AIpert-pwllnod/2021-05-10-AIper...,AIpert-pwllnod,2021-05-10,2021-05-10
64,data-processed/BPagano-RtDriven/2021-05-09-BPa...,BPagano-RtDriven,2021-05-09,2021-05-10
111,data-processed/CEID-Walk/2021-05-10-CEID-Walk.csv,CEID-Walk,2021-05-10,2021-05-10
154,data-processed/CMU-TimeSeries/2021-05-10-CMU-T...,CMU-TimeSeries,2021-05-10,2021-05-10
212,data-processed/COVIDhub-baseline/2021-05-10-CO...,COVIDhub-baseline,2021-05-10,2021-05-10


In [14]:
dfs_test = []

for _, row in tqdm(df_test_files.iterrows(), total=df_test_files.shape[0]):
    #print(row['filename'])
    df_temp = pd.read_csv('https://github.com/reichlab/covid19-forecast-hub/raw/master/' + row['filename'],
                         dtype=dtype, parse_dates=parse_dates)
    df_temp = df_temp[df_temp.target.isin(VALID_TARGETS)]
    df_temp['model'] = row['model']
    dfs_test.append(df_temp)

  0%|          | 0/56 [00:00<?, ?it/s]

In [15]:
df_test = pd.concat(dfs_test)

In [16]:
df_test.shape

(2232941, 8)

In [17]:
df_test

,forecast_date,target,target_end_date,quantile,type,value,location,model
0,2021-05-10,1 wk ahead cum death,2021-05-15,0.025,quantile,584728.816137,US,AIpert-pwllnod
1,2021-05-10,1 wk ahead cum death,2021-05-15,0.250,quantile,585567.867853,US,AIpert-pwllnod
2,2021-05-10,1 wk ahead cum death,2021-05-15,0.750,quantile,586809.830683,US,AIpert-pwllnod
3,2021-05-10,1 wk ahead cum death,2021-05-15,0.975,quantile,588337.761368,US,AIpert-pwllnod
4,2021-05-10,1 wk ahead cum death,2021-05-15,NaN,point,586008.120828,US,AIpert-pwllnod
...,...,...,...,...,...,...,...,...
10939,2021-05-10,4 wk ahead cum death,2021-06-05,0.850,quantile,610855.000000,US,epiforecasts-ensemble1
10940,2021-05-10,4 wk ahead cum death,2021-06-05,0.900,quantile,613494.000000,US,epiforecasts-ensemble1
10941,2021-05-10,4 wk ahead cum death,2021-06-05,0.950,quantile,616718.000000,US,epiforecasts-ensemble1
10942,2021-05-10,4 wk ahead cum death,2021-06-05,0.975,quantile,619093.000000,US,epiforecasts-ensemble1


In [18]:
# only consider US + 50 states
df_test = df_test[df_test.location.str.len() == 2]
df_test = df_test[~df_test.location.isin(locations_to_exclude)]

In [19]:
df_test.shape

(366092, 8)

In [20]:
df_test.location.nunique()

52

In [21]:
# ensure that for all targets each model provides forecasts for all locations
df_test = df_test[df_test.groupby(['target', 'model'])['location'].transform('nunique') == 52]

In [22]:
df_test = df_test[df_test.type == 'quantile']

In [23]:
df_test['no_quantiles'] = df_test.groupby(['model', 'target', 'target_end_date', 'location'])['quantile'].transform('nunique')

df_test['no_quantiles'] = df_test.groupby(['target', 'model'])['no_quantiles'].transform('min')

In [24]:
df_test = df_test[(df_test.no_quantiles == 23) | 
        (df_test.target.str.contains('inc case') & (df_test.no_quantiles == 7))].drop(columns='no_quantiles').reset_index(drop=True)

In [25]:
df_test.shape

(288184, 8)

In [26]:
# dict of the models available for each target
available_models = dict(df_test.groupby(['target'])['model'].unique())

In [27]:
available_models

{'1 wk ahead cum death': array(['BPagano-RtDriven', 'CEID-Walk', 'COVIDhub-baseline', 'CU-select',
        'Covid19Sim-Simulator', 'CovidAnalytics-DELPHI', 'DDS-NBDS',
        'IHME-CurveFit', 'JHUAPL-Bucky', 'JHU_CSSE-DECOM',
        'JHU_IDD-CovidSP', 'Karlen-pypm', 'LANL-GrowthRate', 'LNQ-ens1',
        'MIT_CritData-GBCF', 'Microsoft-DeepSTIA', 'OliverWyman-Navigator',
        'PSI-DRAFT', 'RobertWalraven-ESG', 'SteveMcConnell-CovidComplete',
        'UA-EpiCovDA', 'UCLA-SuEIR', 'UMass-MechBayes', 'USC-SI_kJalpha',
        'epiforecasts-ensemble1'], dtype=object),
 '1 wk ahead inc case': array(['BPagano-RtDriven', 'CEID-Walk', 'COVIDhub-baseline', 'CU-select',
        'Covid19Sim-Simulator', 'CovidAnalytics-DELPHI',
        'IEM_MED-CovidProject', 'JHUAPL-Bucky', 'JHU_CSSE-DECOM',
        'JHU_IDD-CovidSP', 'Karlen-pypm', 'LANL-GrowthRate', 'LNQ-ens1',
        'MUNI-ARIMA', 'Microsoft-DeepSTIA', 'RobertWalraven-ESG',
        'UCLA-SuEIR', 'UChicagoCHATTOPADHYAY-UnIT', 'USC-SI_kJalp

## Training Data

d - 4 weeks - (horizon - 1) --> end: d - horizon

In [28]:
lower_bound = test_date - pd.Timedelta(weeks=4) - pd.Timedelta(weeks=(4 - 1))

In [29]:
lower_bound

Timestamp('2021-03-22 00:00:00')

In [30]:
df_files = df_files[(df_files.timezero >= lower_bound) & (df_files.timezero < test_date)].copy()

In [31]:
# assigns to each horizon the corresponding training forecast dates for the test date
h_dict = {}
for h in range(1, 5):
    h_dict[h] = [test_date - pd.Timedelta(weeks=4) - pd.Timedelta(weeks=(h - 1)), 
                 test_date - pd.Timedelta(weeks=h)]

In [32]:
h_dict

{1: [Timestamp('2021-04-12 00:00:00'), Timestamp('2021-05-03 00:00:00')],
 2: [Timestamp('2021-04-05 00:00:00'), Timestamp('2021-04-26 00:00:00')],
 3: [Timestamp('2021-03-29 00:00:00'), Timestamp('2021-04-19 00:00:00')],
 4: [Timestamp('2021-03-22 00:00:00'), Timestamp('2021-04-12 00:00:00')]}

In [33]:
def relevant_horizons(d):
    hs = []
    for h in range(1, 5):
        if((d >= h_dict[h][0]) & (d <= h_dict[h][1])):
            hs.append(h)
    return hs

In [34]:
df_files['horizons'] = df_files.timezero.apply(relevant_horizons)

In [35]:
df_files

,filename,model,forecast_date,timezero,horizons
13,data-processed/AIpert-pwllnod/2021-03-22-AIper...,AIpert-pwllnod,2021-03-22,2021-03-22,[4]
14,data-processed/AIpert-pwllnod/2021-03-29-AIper...,AIpert-pwllnod,2021-03-29,2021-03-29,"[3, 4]"
15,data-processed/AIpert-pwllnod/2021-04-05-AIper...,AIpert-pwllnod,2021-04-05,2021-04-05,"[2, 3, 4]"
16,data-processed/AIpert-pwllnod/2021-04-12-AIper...,AIpert-pwllnod,2021-04-12,2021-04-12,"[1, 2, 3, 4]"
17,data-processed/AIpert-pwllnod/2021-04-19-AIper...,AIpert-pwllnod,2021-04-19,2021-04-19,"[1, 2, 3]"
...,...,...,...,...,...
3361,data-processed/epiforecasts-ensemble1/2021-04-...,epiforecasts-ensemble1,2021-04-05,2021-04-05,"[2, 3, 4]"
3362,data-processed/epiforecasts-ensemble1/2021-04-...,epiforecasts-ensemble1,2021-04-12,2021-04-12,"[1, 2, 3, 4]"
3363,data-processed/epiforecasts-ensemble1/2021-04-...,epiforecasts-ensemble1,2021-04-19,2021-04-19,"[1, 2, 3]"
3364,data-processed/epiforecasts-ensemble1/2021-04-...,epiforecasts-ensemble1,2021-04-26,2021-04-26,"[1, 2]"


In [36]:
# only keep relevant training data
df_files = df_files[df_files.horizons.apply(len) > 0]

In [37]:
dfs = []
for _, row in tqdm(df_files.iterrows(), total=df_files.shape[0]):
    VALID_TARGETS = [f"{_} wk ahead inc death" for _ in row['horizons']] + \
                    [f"{_} wk ahead cum death" for _ in row['horizons']] + \
                    [f"{_} wk ahead inc case" for _ in row['horizons']]
    df_temp = pd.read_csv('https://github.com/reichlab/covid19-forecast-hub/raw/master/' + row['filename'],
                         dtype=dtype, parse_dates=parse_dates)
    df_temp = df_temp[df_temp.target.isin(VALID_TARGETS)]
    df_temp['model'] = row['model']
    dfs.append(df_temp)

  0%|          | 0/424 [00:00<?, ?it/s]

In [38]:
df = pd.concat(dfs)

In [39]:
df.shape

(9313514, 8)

# Data Cleaning

In [40]:
df = df[df.location.str.len() == 2]
df = df[~df.location.isin(locations_to_exclude)]

In [41]:
df.location.nunique() # US + 50 states + district of Columbia

52

In [42]:
df = df[df.type == 'quantile']

In [43]:
# how many forecasts for each target/model/location? should be 4 for every location
df['no_forecasts'] = df.groupby(['target', 'model', 'location'])['target_end_date'].transform('nunique')

In [44]:
df = df[df.groupby(['target', 'model'])['no_forecasts'].transform('min') == 4].drop(columns='no_forecasts').reset_index(drop=True)

In [45]:
# df = df[df.groupby(['target', 'model', 'location'])['target_end_date'].transform('nunique') == 4]
# would only remove the respective location with <4 forecasts, but we want to remove the model as a whole if in one location <4

In [46]:
df = df[df.groupby(['target', 'model', 'target_end_date'])['location'].transform('nunique') == 52]

In [47]:
df['no_quantiles'] = df.groupby(['model', 'target', 'target_end_date', 'location'])['quantile'].transform('nunique')

df['no_quantiles'] = df.groupby(['target', 'model'])['no_quantiles'].transform('min')

In [48]:
df[(df.target.str.contains('inc case') & (df.no_quantiles == 7))]

,forecast_date,target,target_end_date,quantile,type,value,location,model,no_quantiles
9048,2021-03-21,4 wk ahead inc case,2021-04-17,0.025,quantile,173575.534610,US,BPagano-RtDriven,7
9049,2021-03-21,4 wk ahead inc case,2021-04-17,0.100,quantile,215055.259800,US,BPagano-RtDriven,7
9050,2021-03-21,4 wk ahead inc case,2021-04-17,0.250,quantile,252752.072530,US,BPagano-RtDriven,7
9051,2021-03-21,4 wk ahead inc case,2021-04-17,0.500,quantile,294774.509740,US,BPagano-RtDriven,7
9052,2021-03-21,4 wk ahead inc case,2021-04-17,0.750,quantile,336796.946950,US,BPagano-RtDriven,7
...,...,...,...,...,...,...,...,...,...
1062809,2021-05-03,1 wk ahead inc case,2021-05-08,0.250,quantile,216141.905272,US,UVA-Ensemble,7
1062810,2021-05-03,1 wk ahead inc case,2021-05-08,0.500,quantile,308916.880477,US,UVA-Ensemble,7
1062811,2021-05-03,1 wk ahead inc case,2021-05-08,0.750,quantile,401691.855682,US,UVA-Ensemble,7
1062812,2021-05-03,1 wk ahead inc case,2021-05-08,0.900,quantile,485192.227409,US,UVA-Ensemble,7


In [49]:
df = df[(df.no_quantiles == 23) | 
        (df.target.str.contains('inc case') & (df.no_quantiles == 7))].drop(columns='no_quantiles').reset_index(drop=True)

In [50]:
df.groupby('target').model.nunique()

target
1 wk ahead cum death    20
1 wk ahead inc case     15
1 wk ahead inc death    21
2 wk ahead cum death    19
2 wk ahead inc case     16
2 wk ahead inc death    21
3 wk ahead cum death    19
3 wk ahead inc case     16
3 wk ahead inc death    21
4 wk ahead cum death    20
4 wk ahead inc case     17
4 wk ahead inc death    21
Name: model, dtype: int64

In [51]:
# check if there are models used for training that are not available for the test date
train_models = dict(df.groupby('target').model.unique())

a = [(k, t) for k, v in train_models.items() for t in v]
b = [(k, t) for k, v in available_models.items() for t in v]

[i for i in a if i not in b]

[('2 wk ahead inc death', 'Columbia_UNC-SurvCon'),
 ('3 wk ahead inc death', 'Columbia_UNC-SurvCon'),
 ('4 wk ahead cum death', 'SigSci-TS'),
 ('4 wk ahead inc case', 'SigSci-TS'),
 ('4 wk ahead inc death', 'SigSci-TS')]

In [52]:
df = df[df.apply(lambda x: x.model in (available_models[x.target]), axis=1)]

In [53]:
df.groupby('target').model.nunique()

target
1 wk ahead cum death    20
1 wk ahead inc case     15
1 wk ahead inc death    21
2 wk ahead cum death    19
2 wk ahead inc case     16
2 wk ahead inc death    20
3 wk ahead cum death    19
3 wk ahead inc case     16
3 wk ahead inc death    20
4 wk ahead cum death    19
4 wk ahead inc case     16
4 wk ahead inc death    20
Name: model, dtype: int64

In [54]:
df.shape

(883428, 8)

In [55]:
models_by_target = pd.DataFrame([(i, k) for i,j in train_models.items() for k in j], 
                  columns=['target','model'])

In [56]:
models_by_target[models_by_target.target == '4 wk ahead cum death']

,target,model
168,4 wk ahead cum death,BPagano-RtDriven
169,4 wk ahead cum death,CEID-Walk
170,4 wk ahead cum death,COVIDhub-baseline
171,4 wk ahead cum death,CU-select
172,4 wk ahead cum death,Covid19Sim-Simulator
173,4 wk ahead cum death,CovidAnalytics-DELPHI
174,4 wk ahead cum death,DDS-NBDS
175,4 wk ahead cum death,JHUAPL-Bucky
176,4 wk ahead cum death,Karlen-pypm
177,4 wk ahead cum death,LANL-GrowthRate


In [57]:
df = df[~df.model.isin(models_to_exclude)]

In [58]:
df

,forecast_date,target,target_end_date,quantile,type,value,location,model
0,2021-03-21,4 wk ahead inc death,2021-04-17,0.010,quantile,3200.58484,US,BPagano-RtDriven
1,2021-03-21,4 wk ahead inc death,2021-04-17,0.025,quantile,3650.03224,US,BPagano-RtDriven
2,2021-03-21,4 wk ahead inc death,2021-04-17,0.050,quantile,4051.81453,US,BPagano-RtDriven
3,2021-03-21,4 wk ahead inc death,2021-04-17,0.100,quantile,4522.28842,US,BPagano-RtDriven
4,2021-03-21,4 wk ahead inc death,2021-04-17,0.150,quantile,4841.78172,US,BPagano-RtDriven
...,...,...,...,...,...,...,...,...
904015,2021-05-03,1 wk ahead cum death,2021-05-08,0.850,quantile,582698.00000,US,epiforecasts-ensemble1
904016,2021-05-03,1 wk ahead cum death,2021-05-08,0.900,quantile,582861.00000,US,epiforecasts-ensemble1
904017,2021-05-03,1 wk ahead cum death,2021-05-08,0.950,quantile,583157.00000,US,epiforecasts-ensemble1
904018,2021-05-03,1 wk ahead cum death,2021-05-08,0.975,quantile,583431.00000,US,epiforecasts-ensemble1


In [59]:
available_train_models = dict(df.groupby(['target'])['model'].unique())

In [60]:
available_train_models

{'1 wk ahead cum death': array(['BPagano-RtDriven', 'CEID-Walk', 'COVIDhub-baseline', 'CU-select',
        'Covid19Sim-Simulator', 'CovidAnalytics-DELPHI', 'DDS-NBDS',
        'JHUAPL-Bucky', 'JHU_CSSE-DECOM', 'Karlen-pypm', 'LANL-GrowthRate',
        'LNQ-ens1', 'MIT_CritData-GBCF', 'OliverWyman-Navigator',
        'RobertWalraven-ESG', 'SteveMcConnell-CovidComplete',
        'UA-EpiCovDA', 'UMass-MechBayes', 'USC-SI_kJalpha',
        'epiforecasts-ensemble1'], dtype=object),
 '1 wk ahead inc case': array(['BPagano-RtDriven', 'CEID-Walk', 'COVIDhub-baseline', 'CU-select',
        'Covid19Sim-Simulator', 'CovidAnalytics-DELPHI',
        'IEM_MED-CovidProject', 'JHUAPL-Bucky', 'JHU_CSSE-DECOM',
        'Karlen-pypm', 'LANL-GrowthRate', 'LNQ-ens1', 'RobertWalraven-ESG',
        'USC-SI_kJalpha', 'UVA-Ensemble'], dtype=object),
 '1 wk ahead inc death': array(['BPagano-RtDriven', 'CEID-Walk', 'COVIDhub-baseline', 'CU-select',
        'Covid19Sim-Simulator', 'CovidAnalytics-DELPHI', 'DDS-NB

In [61]:
df_test.shape

(288184, 8)

In [62]:
df_test = df_test[df_test.apply(lambda x: x.model in (available_train_models[x.target]), axis=1)]

In [63]:
df_test.head()

,forecast_date,target,target_end_date,quantile,type,value,location,model
0,2021-05-09,1 wk ahead inc death,2021-05-15,0.010,quantile,2601.47353,US,BPagano-RtDriven
1,2021-05-09,1 wk ahead inc death,2021-05-15,0.025,quantile,2860.84144,US,BPagano-RtDriven
2,2021-05-09,1 wk ahead inc death,2021-05-15,0.050,quantile,3092.70268,US,BPagano-RtDriven
3,2021-05-09,1 wk ahead inc death,2021-05-15,0.100,quantile,3364.20460,US,BPagano-RtDriven
4,2021-05-09,1 wk ahead inc death,2021-05-15,0.150,quantile,3548.57837,US,BPagano-RtDriven


In [64]:
df.groupby(['target'])['model'].unique()

target
1 wk ahead cum death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
1 wk ahead inc case     [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
1 wk ahead inc death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
2 wk ahead cum death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
2 wk ahead inc case     [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
2 wk ahead inc death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
3 wk ahead cum death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
3 wk ahead inc case     [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
3 wk ahead inc death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
4 wk ahead cum death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
4 wk ahead inc case     [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
4 wk ahead inc death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
Name: model, dtype: object

# Export

In [66]:
df.to_csv('data/df_train_{}.csv'.format(str(test_date.date())), index=False)

In [67]:
df_test.to_csv('data/df_test_{}.csv'.format(str(test_date.date())), index=False)